# 第10章: Regression with Panel Data（パネルデータ回帰）練習問題

この章では固定効果モデル、時間効果、パネルデータの標準誤差について学びます。

In [1]:
# 必要なパッケージの読み込み
# install.packages("plm")  # パネルデータ分析用
library(AER)
library(plm)

Loading required package: car



Loading required package: carData



Loading required package: lmtest



Loading required package: zoo




Attaching package: ‘zoo’




The following objects are masked from ‘package:base’:

    as.Date, as.Date.numeric




Loading required package: sandwich



Loading required package: survival



---
## 問題1: パネルデータの構造

Fatalities データ（米国の交通事故死亡データ）を使用します。

1. データを読み込み、構造を確認
2. パネルデータとしての特徴（個体数、時間期間）を確認
3. バランスドパネルかどうかを確認

In [2]:
# 問題1の解答欄



### 模範解答

In [3]:
# 模範解答
data("Fatalities")

# 1. データ構造の確認
cat("=== データの構造 ===", "\n")
str(Fatalities)

# 2. パネルの特徴
cat("\n=== パネルデータの特徴 ===", "\n")
cat("個体（州）の数:", length(unique(Fatalities$state)), "\n")
cat("時間期間:", length(unique(Fatalities$year)), "年\n")
cat("年の範囲:", range(Fatalities$year), "\n")
cat("総観測数:", nrow(Fatalities), "\n")

# 3. バランスドパネルの確認
cat("\n=== バランスドパネルの確認 ===", "\n")
obs_per_state <- table(Fatalities$state)
cat("各州の観測数:\n")
print(table(obs_per_state))
cat("\nバランスドパネル:", length(unique(obs_per_state)) == 1, "\n")

=== データの構造 === 


'data.frame':	336 obs. of  34 variables:
 $ state       : Factor w/ 48 levels "al","az","ar",..: 1 1 1 1 1 1 1 2 2 2 ...
 $ year        : Factor w/ 7 levels "1982","1983",..: 1 2 3 4 5 6 7 1 2 3 ...
 $ spirits     : num  1.37 1.36 1.32 1.28 1.23 ...
 $ unemp       : num  14.4 13.7 11.1 8.9 9.8 ...
 $ income      : num  10544 10733 11109 11333 11662 ...
 $ emppop      : num  50.7 52.1 54.2 55.3 56.5 ...
 $ beertax     : num  1.54 1.79 1.71 1.65 1.61 ...
 $ baptist     : num  30.4 30.3 30.3 30.3 30.3 ...
 $ mormon      : num  0.328 0.343 0.359 0.376 0.393 ...
 $ drinkage    : num  19 19 19 19.7 21 ...
 $ dry         : num  25 23 24 23.6 23.5 ...
 $ youngdrivers: num  0.212 0.211 0.211 0.211 0.213 ...
 $ miles       : num  7234 7836 8263 8727 8953 ...
 $ breath      : Factor w/ 2 levels "no","yes": 1 1 1 1 1 1 1 1 1 1 ...
 $ jail        : Factor w/ 2 levels "no","yes": 1 1 1 1 1 1 1 2 2 2 ...
 $ service     : Factor w/ 2 levels "no","yes": 1 1 1 1 1 1 1 2 2 2 ...
 $ fatal       : int  839


=== パネルデータの特徴 === 


個体（州）の数: 48 


時間期間: 7 年


ERROR: Error in Summary.factor(structure(c(1L, 2L, 3L, 4L, 5L, 6L, 7L, 1L, 2L, : ‘range’ not meaningful for factors


---
## 問題2: プールドOLS

ビール税（beertax）が交通事故死亡率（fatal_rate）に与える影響を分析します。

1. 死亡率変数を作成：fatal_rate = fatal / pop * 10000
2. プールドOLS（個体効果を無視）を推定
3. 結果を解釈

In [4]:
# 問題2の解答欄



### 模範解答

In [5]:
# 模範解答
# 1. 変数の作成
Fatalities$fatal_rate <- Fatalities$fatal / Fatalities$pop * 10000

# 2. プールドOLS
model_pooled <- lm(fatal_rate ~ beertax, data = Fatalities)
cat("=== プールドOLS ===", "\n")
summary(model_pooled)

# 3. 結果の解釈
cat("\n=== 解釈 ===", "\n")
cat("beertaxの係数:", coef(model_pooled)[2], "\n")
cat("→ ビール税が1ドル増加すると、10万人あたりの死亡者数は約",
    round(coef(model_pooled)[2], 2), "人変化\n\n")
cat("注意: この推定は州固有の特性を制御していないため、\n")
cat("      省略変数バイアスの可能性がある\n")

=== プールドOLS === 



Call:
lm(formula = fatal_rate ~ beertax, data = Fatalities)

Residuals:
     Min       1Q   Median       3Q      Max 
-1.09060 -0.37768 -0.09436  0.28548  2.27643 

Coefficients:
            Estimate Std. Error t value Pr(>|t|)    
(Intercept)  1.85331    0.04357  42.539  < 2e-16 ***
beertax      0.36461    0.06217   5.865 1.08e-08 ***
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

Residual standard error: 0.5437 on 334 degrees of freedom
Multiple R-squared:  0.09336,	Adjusted R-squared:  0.09065 
F-statistic: 34.39 on 1 and 334 DF,  p-value: 1.082e-08



=== 解釈 === 


beertaxの係数: 0.3646054 


→ ビール税が1ドル増加すると、10万人あたりの死亡者数は約 0.36 人変化



注意: この推定は州固有の特性を制御していないため、


      省略変数バイアスの可能性がある


---
## 問題3: 固定効果モデル（個体効果）

州固定効果を含むモデルを推定します。

1. plm()関数を使って固定効果モデルを推定
2. 州ダミー変数を使ったlm()による推定と比較
3. プールドOLSとの違いを議論

In [6]:
# 問題3の解答欄



### 模範解答

In [7]:
# 模範解答
# パネルデータオブジェクトの作成
pdata <- pdata.frame(Fatalities, index = c("state", "year"))

# 1. plmによる固定効果モデル
model_fe <- plm(fatal_rate ~ beertax, data = pdata, model = "within")
cat("=== plmによる固定効果モデル ===", "\n")
summary(model_fe)

# 2. lm()による推定（州ダミー使用）
model_lsdv <- lm(fatal_rate ~ beertax + factor(state), data = Fatalities)
cat("\n=== lm()（LSDV）によるbeertax係数 ===", "\n")
cat("係数:", coef(model_lsdv)["beertax"], "\n")

# 3. 比較
cat("\n=== 係数の比較 ===", "\n")
cat("プールドOLS:", coef(model_pooled)[2], "\n")
cat("固定効果モデル:", coef(model_fe)[1], "\n\n")
cat("固定効果を制御すると、beertaxの係数の符号が変わる場合がある。\n")
cat("これは州固有の特性（飲酒文化、道路状況など）を制御した結果。\n")

=== plmによる固定効果モデル === 


Oneway (individual) effect Within Model

Call:
plm(formula = fatal_rate ~ beertax, data = pdata, model = "within")

Balanced Panel: n = 48, T = 7, N = 336

Residuals:
      Min.    1st Qu.     Median    3rd Qu.       Max. 
-0.5869619 -0.0828376 -0.0012701  0.0795454  0.8977960 

Coefficients:
        Estimate Std. Error t-value Pr(>|t|)    
beertax -0.65587    0.18785 -3.4915 0.000556 ***
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

Total Sum of Squares:    10.785
Residual Sum of Squares: 10.345
R-Squared:      0.040745
Adj. R-Squared: -0.11969
F-statistic: 12.1904 on 1 and 287 DF, p-value: 0.00055597


=== lm()（LSDV）によるbeertax係数 === 


係数: -0.6558737 



=== 係数の比較 === 


プールドOLS: 0.3646054 


固定効果モデル: -0.6558737 



固定効果を制御すると、beertaxの係数の符号が変わる場合がある。


これは州固有の特性（飲酒文化、道路状況など）を制御した結果。


---
## 問題4: 時間固定効果

時間（年）固定効果を追加します。

1. 個体固定効果のみのモデル
2. 個体効果+時間効果のモデル（two-way fixed effects）
3. 時間効果の有意性をF検定で確認

In [8]:
# 問題4の解答欄



### 模範解答

In [9]:
# 模範解答
# 1. 個体固定効果のみ
model_fe_entity <- plm(fatal_rate ~ beertax, data = pdata, 
                       model = "within", effect = "individual")

# 2. 個体効果 + 時間効果
model_fe_twoway <- plm(fatal_rate ~ beertax, data = pdata, 
                       model = "within", effect = "twoways")

cat("=== 個体固定効果のみ ===", "\n")
cat("beertax係数:", coef(model_fe_entity), "\n\n")

cat("=== 個体+時間固定効果 ===", "\n")
cat("beertax係数:", coef(model_fe_twoway), "\n\n")

# 3. 時間効果の有意性検定
cat("=== 時間効果のF検定 ===", "\n")
pFtest(model_fe_twoway, model_fe_entity)

# lmによる確認
model_lm_twoway <- lm(fatal_rate ~ beertax + factor(state) + factor(year), 
                      data = Fatalities)
cat("\n年ダミーの結合有意性検定:\n")
library(car)
year_vars <- grep("factor\\(year\\)", names(coef(model_lm_twoway)), value = TRUE)
linearHypothesis(model_lm_twoway, year_vars)

=== 個体固定効果のみ === 


beertax係数: -0.6558737 



=== 個体+時間固定効果 === 


beertax係数: -0.63998 



=== 時間効果のF検定 === 



	F test for twoways effects

data:  fatal_rate ~ beertax
F = 2.0117, df1 = 6, df2 = 281, p-value = 0.0642
alternative hypothesis: significant effects



年ダミーの結合有意性検定:


,Res.Df,RSS,Df,Sum of Sq,F,Pr(>F)
,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
1,287,10.345374,NA,NA,NA,NA
2,281,9.919301,6,0.426073,2.011676,0.06419944


---
## 問題5: クラスターロバスト標準誤差

パネルデータの標準誤差をクラスタリングで修正します。

1. 通常の標準誤差
2. 州でクラスタリングしたロバスト標準誤差
3. 推論への影響を確認

In [10]:
# 問題5の解答欄



### 模範解答

In [11]:
# 模範解答
library(lmtest)

model_fe <- plm(fatal_rate ~ beertax, data = pdata, model = "within")

# 1. 通常の標準誤差
cat("=== 通常の標準誤差 ===", "\n")
print(coeftest(model_fe))

# 2. クラスターロバスト標準誤差
cat("\n=== クラスターロバスト標準誤差（州でクラスタリング）===", "\n")
robust_se <- coeftest(model_fe, vcov = vcovHC(model_fe, type = "HC1", 
                                              cluster = "group"))
print(robust_se)

# 3. 比較
cat("\n=== 標準誤差の比較 ===", "\n")
cat("通常の標準誤差:", summary(model_fe)$coefficients[1, 2], "\n")
cat("クラスターロバスト標準誤差:", robust_se[1, 2], "\n\n")
cat("クラスタリングにより標準誤差が変化し、\n")
cat("推論（t値、p値）にも影響する\n")

=== 通常の標準誤差 === 



t test of coefficients:

        Estimate Std. Error t value Pr(>|t|)    
beertax -0.65587    0.18785 -3.4915 0.000556 ***
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1




=== クラスターロバスト標準誤差（州でクラスタリング）=== 



t test of coefficients:

        Estimate Std. Error t value Pr(>|t|)  
beertax -0.65587    0.28880  -2.271  0.02388 *
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1




=== 標準誤差の比較 === 


通常の標準誤差: 0.18785 


クラスターロバスト標準誤差: 0.2887982 



クラスタリングにより標準誤差が変化し、


推論（t値、p値）にも影響する


---
## 問題6: 変量効果モデル

変量効果（ランダム効果）モデルを推定し、固定効果モデルと比較します。

1. 変量効果モデルを推定
2. Hausman検定を実行
3. 結果に基づいてどちらのモデルが適切か判断

In [12]:
# 問題6の解答欄



### 模範解答

In [13]:
# 模範解答
# 1. 変量効果モデル
model_re <- plm(fatal_rate ~ beertax, data = pdata, model = "random")
cat("=== 変量効果モデル ===", "\n")
summary(model_re)

# 固定効果モデル（比較用）
model_fe <- plm(fatal_rate ~ beertax, data = pdata, model = "within")

# 係数の比較
cat("\n=== 係数の比較 ===", "\n")
cat("固定効果モデル:", coef(model_fe), "\n")
cat("変量効果モデル:", coef(model_re)[2], "\n\n")

# 2. Hausman検定
cat("=== Hausman検定 ===", "\n")
hausman_test <- phtest(model_fe, model_re)
print(hausman_test)

# 3. 判断
cat("\n=== 結論 ===", "\n")
if (hausman_test$p.value < 0.05) {
  cat("p値 < 0.05：帰無仮説（変量効果が適切）を棄却\n")
  cat("→ 固定効果モデルが適切\n")
} else {
  cat("p値 >= 0.05：帰無仮説を棄却できない\n")
  cat("→ 変量効果モデルも使用可能（より効率的）\n")
}

=== 変量効果モデル === 


Oneway (individual) effect Random Effect Model 
   (Swamy-Arora's transformation)

Call:
plm(formula = fatal_rate ~ beertax, data = pdata, model = "random")

Balanced Panel: n = 48, T = 7, N = 336

Effects:
                  var std.dev share
idiosyncratic 0.03605 0.18986 0.119
individual    0.26604 0.51579 0.881
theta: 0.8622

Residuals:
     Min.   1st Qu.    Median   3rd Qu.      Max. 
-0.471090 -0.120045 -0.021530  0.091011  0.964350 

Coefficients:
             Estimate Std. Error z-value Pr(>|z|)    
(Intercept)  2.067141   0.099971 20.6773   <2e-16 ***
beertax     -0.052016   0.124176 -0.4189   0.6753    
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

Total Sum of Squares:    12.648
Residual Sum of Squares: 12.642
R-Squared:      0.00052508
Adj. R-Squared: -0.0024674
Chisq: 0.175467 on 1 DF, p-value: 0.6753


=== 係数の比較 === 


固定効果モデル: -0.6558737 


変量効果モデル: -0.0520158 



=== Hausman検定 === 



	Hausman Test

data:  fatal_rate ~ beertax
chisq = 18.353, df = 1, p-value = 1.835e-05
alternative hypothesis: one model is inconsistent




=== 結論 === 


p値 < 0.05：帰無仮説（変量効果が適切）を棄却
→ 固定効果モデルが適切


---
## 問題7: コントロール変数の追加

他の説明変数を追加したモデルを推定します。

1. fatal_rate ~ beertax + drinkage + miles + income のモデルを推定
2. 固定効果モデル（two-way）で推定
3. 各係数を解釈

In [14]:
# 問題7の解答欄



### 模範解答

In [15]:
# 模範解答
# drinkage21: 飲酒年齢21歳のダミー変数を作成
Fatalities$drinkage21 <- ifelse(Fatalities$drinkage >= 21, 1, 0)
pdata <- pdata.frame(Fatalities, index = c("state", "year"))

# 1-2. 複数の説明変数を含む固定効果モデル
model_full <- plm(fatal_rate ~ beertax + drinkage + miles + income,
                  data = pdata, model = "within", effect = "twoways")

cat("=== 固定効果モデル（two-way）===", "\n")
summary(model_full)

# クラスターロバスト標準誤差
cat("\n=== クラスターロバスト標準誤差 ===", "\n")
coeftest(model_full, vcov = vcovHC(model_full, type = "HC1", cluster = "group"))

# 3. 解釈
cat("\n=== 係数の解釈 ===", "\n")
cat("beertax: ビール税が1ドル上昇すると死亡率が", 
    round(coef(model_full)["beertax"], 3), "変化\n")
cat("drinkage: 飲酒可能年齢が1歳上昇すると死亡率が",
    round(coef(model_full)["drinkage"], 3), "変化\n")
cat("miles: 平均走行距離が増加すると死亡率が",
    round(coef(model_full)["miles"], 6), "変化\n")
cat("income: 所得が増加すると死亡率が",
    round(coef(model_full)["income"], 6), "変化\n")

=== 固定効果モデル（two-way）=== 


Twoways effects Within Model

Call:
plm(formula = fatal_rate ~ beertax + drinkage + miles + income, 
    data = pdata, effect = "twoways", model = "within")

Balanced Panel: n = 48, T = 7, N = 336

Residuals:
      Min.    1st Qu.     Median    3rd Qu.       Max. 
-0.4891887 -0.0897612  0.0067138  0.0885985  0.6054979 

Coefficients:
            Estimate  Std. Error t-value  Pr(>|t|)    
beertax  -5.6490e-01  1.8178e-01 -3.1076  0.002082 ** 
drinkage  2.4315e-03  1.9656e-02  0.1237  0.901638    
miles     1.2140e-05  9.6822e-06  1.2539  0.210948    
income    1.4884e-04  2.0393e-05  7.2986 3.059e-12 ***
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

Total Sum of Squares:    10.29
Residual Sum of Squares: 8.2342
R-Squared:      0.19982
Adj. R-Squared: 0.035758
F-statistic: 17.3558 on 4 and 278 DF, p-value: 1.0044e-12


=== クラスターロバスト標準誤差 === 



t test of coefficients:

            Estimate  Std. Error t value Pr(>|t|)   
beertax  -5.6490e-01  3.4740e-01 -1.6260 0.105073   
drinkage  2.4315e-03  2.6212e-02  0.0928 0.926157   
miles     1.2140e-05  8.7252e-06  1.3914 0.165222   
income    1.4884e-04  4.5224e-05  3.2912 0.001126 **
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1



=== 係数の解釈 === 


beertax: ビール税が1ドル上昇すると死亡率が -0.565 変化


drinkage: 飲酒可能年齢が1歳上昇すると死亡率が 0.002 変化


miles: 平均走行距離が増加すると死亡率が 1.2e-05 変化


income: 所得が増加すると死亡率が 0.000149 変化


---
## 問題8: 差分の差分（First Differences）

1階差分モデルを推定します。

1. plmのmodel="fd"オプションで1階差分モデルを推定
2. 固定効果モデルとの比較
3. どちらが適切か議論

In [16]:
# 問題8の解答欄



### 模範解答

In [17]:
# 模範解答
# 1. 1階差分モデル
model_fd <- plm(fatal_rate ~ beertax, data = pdata, model = "fd")
cat("=== 1階差分モデル ===", "\n")
summary(model_fd)

# 2. 固定効果モデル（比較用）
model_fe <- plm(fatal_rate ~ beertax, data = pdata, model = "within")

cat("\n=== 係数の比較 ===", "\n")
cat("1階差分モデル:", coef(model_fd), "\n")
cat("固定効果モデル:", coef(model_fe), "\n\n")

# 3. 議論
cat("=== モデル選択の議論 ===", "\n")
cat("固定効果モデル:\n")
cat("  - 個体内の全ての時点の情報を使用\n")
cat("  - 誤差項が系列相関を持たない場合に効率的\n\n")
cat("1階差分モデル:\n")
cat("  - 隣接する時点の差のみを使用\n")
cat("  - 誤差項がランダムウォークの場合に効率的\n")
cat("  - T=2の場合は両モデルは同一\n")

=== 1階差分モデル === 


Oneway (individual) effect First-Difference Model

Call:
plm(formula = fatal_rate ~ beertax, data = pdata, model = "fd")

Balanced Panel: n = 48, T = 7, N = 336
Observations used in estimation: 288

Residuals:
     Min.   1st Qu.    Median   3rd Qu.      Max. 
-0.697391 -0.106403  0.010073  0.109465  0.606420 

Coefficients:
              Estimate Std. Error t-value Pr(>|t|)
(Intercept) -0.0031368  0.0119115 -0.2633   0.7925
beertax      0.0136878  0.2852511  0.0480   0.9618

Total Sum of Squares:    11.213
Residual Sum of Squares: 11.213
R-Squared:      8.0509e-06
Adj. R-Squared: -0.0034884
F-statistic: 0.00230256 on 1 and 286 DF, p-value: 0.96176


=== 係数の比較 === 


1階差分モデル: -0.003136839 0.01368779 


固定効果モデル: -0.6558737 



...

  - 誤差項が系列相関を持たない場合に効率的



1階差分モデル:


  - 隣接する時点の差のみを使用


  - 誤差項がランダムウォークの場合に効率的


  - T=2の場合は両モデルは同一


---
## 問題9: 動学パネルモデル

被説明変数のラグを含むモデルを検討します。

1. fatal_rate(t-1)をラグ変数として含むモデルを推定
2. 動学パネルにおけるバイアスの問題を議論
3. 注意点を説明

In [18]:
# 問題9の解答欄



### 模範解答

In [19]:
# 模範解答
# ラグ変数を作成
pdata$fatal_rate_lag <- lag(pdata$fatal_rate, 1)

# 1. ラグ変数を含むモデル（注意：バイアスあり）
model_dynamic <- plm(fatal_rate ~ fatal_rate_lag + beertax, 
                     data = pdata, model = "within")
cat("=== 動学パネルモデル（固定効果）===", "\n")
cat("注意：このモデルにはNickell biasが存在する可能性\n\n")
summary(model_dynamic)

# 2-3. バイアスの議論
cat("\n=== 動学パネルの問題点 ===", "\n")
cat("Nickell bias（動学パネルバイアス）:\n")
cat("  - 固定効果モデルでラグ付き被説明変数を含めると\n")
cat("  - 変換後の誤差項とラグ変数が相関\n")
cat("  - → OLS推定量にバイアスが生じる\n\n")
cat("対処法:\n")
cat("  - 操作変数法（Arellano-Bond推定量など）\n")
cat("  - 長いパネル（Tが大きい）の場合はバイアスが小さい\n")
cat("  - バイアス補正推定量の使用\n")

=== 動学パネルモデル（固定効果）=== 


注意：このモデルにはNickell biasが存在する可能性



Oneway (individual) effect Within Model

Call:
plm(formula = fatal_rate ~ fatal_rate_lag + beertax, data = pdata, 
    model = "within")

Balanced Panel: n = 48, T = 6, N = 288

Residuals:
      Min.    1st Qu.     Median    3rd Qu.       Max. 
-0.4938899 -0.0789033  0.0081699  0.0812261  0.5952727 

Coefficients:
                Estimate Std. Error t-value  Pr(>|t|)    
fatal_rate_lag  0.291038   0.052089  5.5874 6.286e-08 ***
beertax        -0.338533   0.213664 -1.5844    0.1144    
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

Total Sum of Squares:    6.4367
Residual Sum of Squares: 5.5129
R-Squared:      0.14352
Adj. R-Squared: -0.032818
F-statistic: 19.9403 on 2 and 238 DF, p-value: 9.852e-09


=== 動学パネルの問題点 === 


Nickell bias（動学パネルバイアス）:


...

  - → OLS推定量にバイアスが生じる



対処法:


  - 操作変数法（Arellano-Bond推定量など）


  - 長いパネル（Tが大きい）の場合はバイアスが小さい


  - バイアス補正推定量の使用


---
## 問題10: 総合分析

飲酒規制政策の効果を総合的に分析します。

1. 複数のモデル仕様で推定
2. 結果を比較表にまとめる
3. 政策的含意を議論

In [20]:
# 問題10の解答欄



### 模範解答

In [21]:
# 模範解答
# 複数のモデルを推定
m1 <- lm(fatal_rate ~ beertax, data = Fatalities)  # Pooled OLS
m2 <- plm(fatal_rate ~ beertax, data = pdata, model = "within")  # FE（個体）
m3 <- plm(fatal_rate ~ beertax, data = pdata, model = "within", effect = "twoways")  # FE（双方向）
m4 <- plm(fatal_rate ~ beertax + drinkage + income, data = pdata, 
          model = "within", effect = "twoways")  # FE + コントロール

# 結果の比較
cat("=== モデル比較 ===", "\n\n")
cat("                   Pooled   FE(ind)  FE(2way) FE+Ctrl\n")
cat("--------------------------------------------------\n")
cat(sprintf("beertax        %8.4f %8.4f %8.4f %8.4f\n",
            coef(m1)[2], coef(m2), coef(m3), coef(m4)["beertax"]))

# 標準誤差（クラスターロバスト）
se_m2 <- sqrt(vcovHC(m2, type = "HC1", cluster = "group")[1,1])
se_m3 <- sqrt(vcovHC(m3, type = "HC1", cluster = "group")[1,1])
se_m4 <- sqrt(vcovHC(m4, type = "HC1", cluster = "group")["beertax", "beertax"])

cat(sprintf("SE(cluster)    %8s %8.4f %8.4f %8.4f\n",
            "--", se_m2, se_m3, se_m4))

# t値
cat(sprintf("t-stat         %8.2f %8.2f %8.2f %8.2f\n",
            coef(m1)[2]/summary(m1)$coefficients[2,2],
            coef(m2)/se_m2, coef(m3)/se_m3, coef(m4)["beertax"]/se_m4))

cat("\n=== 政策的含意 ===", "\n")
cat("1. プールドOLSでは正の関係（偽相関の可能性）\n")
cat("2. 固定効果モデルでは負の関係（ビール税上昇→死亡率減少）\n")
cat("3. 双方向固定効果でも同様の傾向\n")
cat("4. コントロール変数を追加しても結果は頑健\n\n")
cat("結論: ビール税の引き上げは交通事故死亡率を\n")
cat("      減少させる効果がある可能性が示唆される\n")

=== モデル比較 === 



                   Pooled   FE(ind)  FE(2way) FE+Ctrl


--------------------------------------------------


beertax          0.3646  -0.6559  -0.6400  -0.5864


SE(cluster)          --   0.2888   0.3501   0.3456


...

2. 固定効果モデルでは負の関係（ビール税上昇→死亡率減少）


3. 双方向固定効果でも同様の傾向


4. コントロール変数を追加しても結果は頑健



結論: ビール税の引き上げは交通事故死亡率を


      減少させる効果がある可能性が示唆される
